# Required imports

In [1]:
import os
import json
import matplotlib.pyplot as plt
import pandas as pd

from helper_functions import *
from hk_news_features import *

# Plot settings

In [2]:
SIZE = 10

plt.rc('xtick', labelsize=SIZE)
plt.rc('ytick', labelsize=SIZE)
plt.rc('font', size=SIZE)
plt.rc('axes', labelsize=SIZE)
# plt.rcParams["figure.figsize"] = (15,11)


metrics_palette = {
    'comet22': "lightsteelblue", 
    'comet22-qe': "royalblue", 
    'chrf': "khaki", 
    'sacrebleu': "goldenrod",
}

scalers_palette = {
    'none, lin': "#11324D",
    'none, rf': "#6B7AA1", 
    'none, xgb': "#A6DCEF",
    'standard, lin': "#3A4D39",
    'standard, rf': "#79AC78", 
    'standard, xgb': "#D0E7D2", 
    'min-max, lin': "#7F669D",
    'min-max, rf': "#BA94D1", 
    'min-max, xgb': "#DEBACE",
}

%matplotlib widget

# Load data

In [3]:
BASE_DIR = f"{os.getcwd()}/../../"
MODEL_EVAL_DIR = BASE_DIR + "model_eval/"

In [4]:
fid_file = "fid-finetune_data-dataset_hk-news-timeInterval_1-timeIntervalType_time-finetuneType_base.csv"
fid = pd.read_csv(MODEL_EVAL_DIR + fid_file)

fid = set_type_of_time_cols(fid)

In [ ]:
fid.columns

## Add delta target features

In [6]:
for metric in METRICS:
    for tactic in TACTICS:
        fid[f'delta-target_{metric}_{tactic}'] = fid[f'target_{metric}_{tactic}'] - fid[f'curr_{metric}']

    for test_set in HK_NEWS_TEST_SETS:
        key = f"{test_set}_{metric}"
        fid[f"delta-target_{key}"] = fid[f"target_{key}"] - fid[f"curr_{key}"]

## View data

In [ ]:
fid[[
    'curr_finetune',
    'prev_finetune',
    'period_start_timestamp',
    'period_end_timestamp',
    'prev_finetune_period_start_timestamp',
    'prev_finetune_period_end_timestamp',
    'curr_finetune_week',
    'prev_finetune_week',
    'amount_new_data',
]]

### Look for NaN

In [ ]:
print("Feature name")
print("-"*50)
for col in fid.columns:
    print(f"{col}\t\t{fid[col].isna().sum()}")

# Feature correlation

In [ ]:
features = BASIC_FEATURES + ['delta-target_Finance_comet22']

make_corr(features, fid, "Feature correlation")

# Build and evaluate predictors

## Predictors

In [ ]:
predictors = [
    "lin",
    "rf", 
    "xgb",
]
f"predictors: {predictors}"

## Features dict

In [ ]:
features_dict = {
    "All": BASIC_FEATURES + SYS_PERF_FEATURES + CONTENT_AWARE_FEATURES,
    "Basic": BASIC_FEATURES,
    "ContAware": CONTENT_AWARE_FEATURES,
    "MTQual": SYS_PERF_FEATURES,
    "Basic-MTQual": BASIC_FEATURES + SYS_PERF_FEATURES,
    "Basic-ContAware": BASIC_FEATURES + CONTENT_AWARE_FEATURES,
    "ContAware--no-ngrams": (
        SENT_OVERLAP_FEATURES["new_data"] + EMBEDDING_FEATURES["new_data"]
        + SENT_OVERLAP_FEATURES["finetune_data"] + EMBEDDING_FEATURES["finetune_data"]
    ),
    "ContAware-MTQual": CONTENT_AWARE_FEATURES + SYS_PERF_FEATURES,
    "All-kiwi": BASIC_FEATURES + CONTENT_AWARE_FEATURES + [f for f in SYS_PERF_FEATURES if '-qe' in f],
    "Basic-kiwi": BASIC_FEATURES + [f for f in SYS_PERF_FEATURES if '-qe' in f],
    "ContAware-kiwi": CONTENT_AWARE_FEATURES + [f for f in SYS_PERF_FEATURES if '-qe' in f],
}
print("features_dict:")
print(json.dumps(features_dict, indent=4))

In [12]:
res_dict = {}
feature_imp_dict = {}

In [ ]:
for metric in ["comet22", "chrf", "sacrebleu", "comet22-qe"]:
    for test_set in HK_NEWS_TEST_SETS:
        target = f'delta-target_{test_set}_{metric}'

        eval_FIPs_offline(
            target=target, 
            features_dict=features_dict, 
            predictors=predictors, 
            res_dict=res_dict, 
            feature_imp_dict=feature_imp_dict, 
            dataset=fid,
            dataset_name='hk-news',
            fip_type='specific',
            fid_type='normal'
        )

## Results

In [14]:
results = pd.DataFrame(res_dict).transpose()
results['PCC'] = results['test-PCC'] * 100
results['MAE'] = results['test-mae']

results = results.round({'PCC': 2, 'MAE': 4})

### COMET22

In [ ]:
get_results_table(
    results = results.loc[results.metric=='comet22'],
    fid_type = 'normal',
    average = False,
    single = False,    # whether to return a single table or a table for each predictor
    to_latex = True,  # whether to print the table in latex
)

### chrF

In [ ]:
get_results_table(
    results = results.loc[results.metric=='chrf'],
    fid_type = 'normal',
    average = False,
    single = False,    # whether to return a single table or a table for each predictor
    to_latex = True,  # whether to print the table in latex
)

### sacreBLEU

In [ ]:
get_results_table(
    results = results.loc[results.metric=='sacrebleu'],
    fid_type = 'normal',
    average = False,
    single = False,    # whether to return a single table or a table for each predictor
    to_latex = True,  # whether to print the table in latex
)

### comet22-qe

In [ ]:
get_results_table(
    results = results.loc[results.metric=='comet22-qe'],
    fid_type = 'normal',
    average = False,
    single = False,    # whether to return a single table or a table for each predictor
    to_latex = True,  # whether to print the table in latex
)